## Loading Dataset

In [2]:
import psycopg2
import pandas as pd
conn = psycopg2.connect("dbname=musicdatabase")
cur = conn.cursor()

## Creating DataFrames

In [3]:
#GENRES:
cur.execute("SELECT * FROM genres")
names = [description[0] for description in cur.description]
genres = cur.fetchall()
genres_df = pd.DataFrame(genres, columns=names)

# #ARTISTS
cur.execute("SELECT * FROM artists")
names = [description[0] for description in cur.description]
artists = cur.fetchall()
artists_df = pd.DataFrame(artists, columns=names)

# #CONTENT
cur.execute("SELECT * FROM content")
names = [description[0] for description in cur.description]
content = cur.fetchall()
content_df = pd.DataFrame(content, columns=names)

# #LABELS
cur.execute("SELECT * FROM labels")
names = [description[0] for description in cur.description]
labels = cur.fetchall()
labels_df = pd.DataFrame(labels, columns=names)

# #REVIEWS
cur.execute("SELECT * FROM reviews")
names = [description[0] for description in cur.description]
reviews = cur.fetchall()
reviews_df = pd.DataFrame(reviews, columns=names)

# #YEARS
cur.execute("SELECT * FROM years")
names = [description[0] for description in cur.description]
years = cur.fetchall()
years_df = pd.DataFrame(years, columns=names)

## Exploring the Data

In [4]:
print('genre df length: ', len(genres_df))
print('artists df length: ', len(artists_df))
print('content df length: ', len(content_df))
print('labels df length: ', len(labels_df))
print('reviews df length: ', len(reviews_df))
print('years df length: ', len(years_df))

genre df length:  22680
artists df length:  18831
content df length:  18393
labels df length:  20190
reviews df length:  18393
years df length:  19108


In [5]:
reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18393 entries, 0 to 18392
Data columns (total 13 columns):
reviewid          18393 non-null int64
title             18393 non-null object
artist            18393 non-null object
url               18393 non-null object
score             18393 non-null float64
best_new_music    18393 non-null int64
author            18393 non-null object
author_type       14487 non-null object
pub_date          18393 non-null object
pub_weekday       18393 non-null int64
pub_day           18393 non-null int64
pub_month         18393 non-null int64
pub_year          18393 non-null int64
dtypes: float64(1), int64(6), object(6)
memory usage: 1.8+ MB


In [6]:
reviews_df.isna().sum()
#no NANs in column 1.. however 2367 in column 2

reviewid             0
title                0
artist               0
url                  0
score                0
best_new_music       0
author               0
author_type       3906
pub_date             0
pub_weekday          0
pub_day              0
pub_month            0
pub_year             0
dtype: int64

## Scratch Work & More Review

In [7]:
reviews_df['author'].value_counts()
reviews_df['author'].nunique()

432

In [8]:
reviews_df['artist'].value_counts().head()

various artists     688
guided by voices     23
the beatles          21
david bowie          21
mogwai               20
Name: artist, dtype: int64

In [9]:
len(set(reviews_df['reviewid']))
#reviews_df['reviewid'].nunique()  #does the same thing

18389

## Seeing Which Tables Have Multiple Rows Per Song

In [10]:
genres_df.loc[genres_df['reviewid'] == 11733]
#example, song 6523 has 4 artists

,reviewid,genre
15417,11733,rock
15418,11733,global
15419,11733,experimental
15420,11733,electronic


* Review Table -- has 4 songs that have 2 reviewid's... i believe this is an error as all rows are identical
* Content Table -- also has 4 songs with 2 reviewid's... will remove the duplicated rows
* Genres Tabel -- Many songs have 2+ genres
* Labels Tabel -- Many songs have 2+ labels
* Artists Tabel -- Many songs have 2+ artists
* Year Tabel -- Many songs have 2+ release years... this may be an error... considering keeping only oldest year

In [11]:
genres_df['reviewid'].value_counts().head(10)

11733    4
21701    4
7683     4
757      4
12222    4
14812    4
12625    4
22465    4
9530     4
9036     4
Name: reviewid, dtype: int64

## Combining Genre and Review Tables

First, one hot encode genres table. 

In [66]:
# genres_df['pop/r&b'] = genres_df['genre'].apply(lambda x: 1 if x == 'pop/r&b' else 0)
# genres_df['electronic'] = genres_df['genre'].apply(lambda x: 1 if x == 'electronic' else 0)

In [12]:
genres_one_hot = pd.get_dummies(genres_df['genre'])
genres_one_hot.head()
genres_hot_df = pd.concat([genres_df, genres_one_hot], axis = 1)
genres_hot_df.head()

,reviewid,genre,electronic,experimental,folk/country,global,jazz,metal,pop/r&b,rap,rock
0,22703,electronic,1,0,0,0,0,0,0,0,0
1,22721,metal,0,0,0,0,0,1,0,0,0
2,22659,rock,0,0,0,0,0,0,0,0,1
3,22661,rock,0,0,0,0,0,0,0,0,1
4,22725,electronic,1,0,0,0,0,0,0,0,0


Second, filtered genres table for only pop & electric

In [13]:
genres_hot_df_filtered = genres_hot_df[(genres_hot_df['pop/r&b'] == 1) | (genres_hot_df['electronic'] == 1)]
genres_hot_df_filtered = genres_hot_df_filtered[['reviewid', 'electronic', 'pop/r&b']]
genres_hot_df_filtered.head()

,reviewid,electronic,pop/r&b
0,22703,1,0
4,22725,1,0
7,22694,1,0
12,22700,1,0
18,22719,0,1


Third, add value_count column in order to remove any albums that fall under both electronic & pop.

In [14]:
z = genres_hot_df_filtered['reviewid'].value_counts()
z1 = z.to_dict()
genres_hot_df_filtered['value_count'] = genres_hot_df_filtered['reviewid'].apply(lambda x: z1[x])
genres_hot_df_filtered.head()

,reviewid,electronic,pop/r&b,value_count
0,22703,1,0,1
4,22725,1,0,1
7,22694,1,0,1
12,22700,1,0,1
18,22719,0,1,1


Remove albums with both electronic and pop (where value_counts = 2)

In [15]:
to_drop = genres_hot_df_filtered[genres_hot_df_filtered['value_count'] == 2]
genres_df = genres_hot_df_filtered.drop(to_drop.index, axis=0)
genres_df.head()

,reviewid,electronic,pop/r&b,value_count
0,22703,1,0,1
4,22725,1,0,1
7,22694,1,0,1
12,22700,1,0,1
18,22719,0,1,1


Merge the new Genre Table with the existing Review Table

In [17]:
#rg = review & genres
rg_df = reviews_df.merge(genres_df, left_on='reviewid', right_on='reviewid')
rg_df

,reviewid,title,artist,url,score,best_new_music,author,author_type,pub_date,pub_weekday,pub_day,pub_month,pub_year,electronic,pop/r&b,value_count
0,22703,mezzanine,massive attack,http://pitchfork.com/reviews/albums/22703-mezz...,9.3,0,nate patrin,contributor,2017-01-08,6,8,1,2017,1,0,1
1,22725,new start,taso,http://pitchfork.com/reviews/albums/22725-new-...,8.1,0,kevin lozano,tracks coordinator,2017-01-06,4,6,1,2017,1,0,1
2,22694,tehillim,yotam avni,http://pitchfork.com/reviews/albums/22694-tehi...,7.0,0,andy beta,contributor,2017-01-05,3,5,1,2017,1,0,1
3,22700,nadir,steven warwick,http://pitchfork.com/reviews/albums/22700-nadir/,7.6,0,thea ballard,contributor,2017-01-03,1,3,1,2017,1,0,1
4,22719,merry christmas lil mama,"chance the rapper, jeremih",http://pitchfork.com/reviews/albums/22719-merr...,8.1,0,sheldon pearce,associate staff writer,2016-12-30,4,30,12,2016,0,1,1
5,22691,love you to death,the-dream,http://pitchfork.com/reviews/albums/22691-love...,7.0,0,rebecca haithcoat,contributor,2016-12-29,3,29,12,2016,0,1,1
6,22713,the wasp factory,ben frost,http://pitchfork.com/reviews/albums/22713-the-...,7.3,0,brian howe,contributor,2016-12-27,1,27,12,2016,1,0,1
7,22701,traditional music of notional species vol. ii,rashad becker,http://pitchfork.com/reviews/albums/22701-trad...,6.8,0,andy beta,contributor,2016-12-23,4,23,12,2016,1,0,1
8,22685,in the loop: a decade of remixes,a-trak,http://pitchfork.com/reviews/albums/22685-in-t...,6.4,0,andy beta,contributor,2016-12-17,5,17,12,2016,1,0,1
9,22681,w:/2016album/,deadmau5,http://pitchfork.com/reviews/albums/22681-w201...,5.9,0,philip sherburne,contributing editor,2016-12-16,4,16,12,2016,1,0,1
